# An Introduction to Recurrent Neural Networks

Mike Heilman ([@heilman13](https://twitter.com/heilman13))

[Civis Analytics](https://civisanalytics.com/) ([@CivisAnalytics](https://twitter.com/CivisAnalytics))

November, 2016

## Background

This is a little intro about RNNs I gave to folks at Civis.  I figured I might as well share it publicly.

There are quite a few sources of info on RNNs (links below), but maybe one more variation will help.

I'll try to provide some links, but this is by no means a comprehensive literature review!

## What to expect

We're going to start from very simple models and build up to complex recurrent neural networks.

Then, we'll talk about some modeling scenarios and applications.


## Notation

* $X$: input
  * shape: (n_examples, n_features)
  * aka features, independent variables, predictors
* $Y$: outputs
  * shape: (n_examples, n_targets)
  * aka labels, targets, dependent variables
  * I'll assume the targets are discrete and 1-hot encoded here (e.g., words in a vocabulary).
* $W$: the parameters for the model
  * a.k.a. weights, coefficients
  * This is usually many matrices. I'll use subscripts when needed.
  * typically, shape: (layer_input_size, layer_output_size)
  * I'll also use $U$ later on.

## Linear Regression

$ Y = XW $

<img src="nn_diagrams_logistic.svg" width="100px"/>

### Notes

* There's often an intercept/bias term $b$ added at the end of the above eq., but I'm leaving it out for simplicity.
* Arrows represent transformations (multiplication by a matrix of weight parameters).

## Logistic Regression

### Multiclass

$ p(Y = y \mid X, W) = \frac{\exp(XW_{y})}{ \displaystyle\sum_{y' \in \mathcal{Y}}{\exp(XW_{y'})}} = \textrm{softmax}(XW)_y $


### Binary

(people usually let one class be implicit and simplify)

$ p(Y = 1 \mid X, W) = \frac{\exp(XW)}{ \exp(XW) + \exp(0)} = \frac{1}{ 1 + \exp(-XW)} = \textrm{sigmoid}(XW) $

<img src="nn_diagrams_logistic.svg" width="100px"/>


## Categorical outputs.

Subsequently, for consistency, I'll assume we have categorical outputs and use a softmax activation function.

RNNs can work with other types of outputs (e.g., binary) as with linear and logistic regression.

## Optimization (finding good parameter values)

* Minimize the **negative log likelihood** of the parameters given the observed data (true labels).
  * Equivalently, minimize the **cross entropy** between the distribution of the model's predicted labels and the true distribution.
  * Or, e.g., mean squared error for linear regression with continuous data.
* To do this, we can use algorithms based on stochastic gradient descent (e.g., Adagrad, Adam).


## Multilayer Perceptron

$ P(Y = y \mid X, W) = \textrm{softmax}(ZW_{out})_y $

$ Z = \tanh(XW_{h}) $

<img src="nn_diagrams_mlp.svg" width="150px"/>

### Notes

* Instead of tanh, we could use some other nonlinear function (e.g., sigmoid, ReLU).
* For non-categorical data, we could use some output activation other than the $\textrm{softmax}$ function.


## Simple RNN


$ P(Y_t = y \mid \ldots) = \textrm{softmax}(S_t W_{out})_y $

$ S_t = f(X W_{in} + S_{t-1} W_{recur}) $


<img src="nn_diagrams_simple_rnn.svg" width="500px"/>


## Stacked RNNs

In order to (maybe) do a better job at model interactions, we can stack RNNs similar to adding hidden layers in a multilayer perceptron:


$ P(Y_t = y \mid \ldots) = \textrm{softmax}(S_{t,1} W_{out})_y $

$ S_{t,1} = f(S_{t,0} W_{in,1} + S_{t-1,1} W_{recur,1}) $

$ S_{t,0} = f(X W_{in,0} + S_{t-1,0} W_{recur,0}) $

<img src="nn_diagrams_stacked_rnn.svg" width="200px"/>


## Learning long distance dependencies

* Recall that the RNN state is a combination of the previous state and new input:

$ S_t = f(X W_{in} + S_{t-1} W_{recur}) $

* As more input is read in, attributing information in the state to particular inputs and weights becomes difficult.



## Gates

* In Gated RNNs and Long Short Term Memory (LSTM) networks, **gates** regulate what information is retained in and added to the current state
  * Gates depend on the previous state $s_{t-1}$ and the current input $x_t$.
  * For example, gates could help ignore the repetition of particular information ("very very very good").

**Related:** vanishing and exploding gradients. See, e.g., [this](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/).


## Long Short Term Memory (LSTM) recurrent networks

### Simplified version

Input, forget, and output gates:

$ i = \textrm{sigmoid}(x_t U^{i} + s_{t-1} W^{i}) $

$ f = \textrm{sigmoid}(x_t U^{f} + s_{t-1} W^{f}) $

$ o = \textrm{sigmoid}(x_t U^{o} + s_{t-1} W^{o}) $

State update (simplified! see below):

$ c_{t} = c_{t-1} \circ f + X_t \circ i $

**Note:** "$\circ$" denotes elementwise multiplication.



### Full version

Actually, LSTMs are a bit more complicated.



Transformed input:

$ g = \tanh(x_t U^{g} + s_{t-1} W^{g}) $

State update:

$ c_{t} = c_{t-1} \circ f + g \circ i $

Output:

$ s_t = \tanh(c_t) \circ o $

**Note:** unlike the simple RNN, the LSTM output is not just the current state's value.



### Variations

People have tried lots of variations of the LSTM architecture.  See, e.g., [this paper](http://arxiv.org/pdf/1503.04069.pdf).

Also, there are [Gated RNNs](https://arxiv.org/abs/1412.3555):

* fewer parameters than LSTMs
* often perform as well as or better than LSTMs

## Embeddings for sparse data

* High dimensional inputes (e.g., words) are often embedded into lower dimensions before being used as network input.
  * e.g., 100,000 word indicators $\rightarrow$ 300 dimensions
  * This keeps the RNN inputs (and gates) from having very high dimensionality.
  
### Pretrained embeddings

* Embeddings are sometimes separately pretrained on large amounts of unlabeled in-domain data.
  * e.g., [word2vec](https://radimrehurek.com/gensim/models/word2vec.html) or [GloVe](http://nlp.stanford.edu/projects/glove/) embeddings trained on Wikipedia.
* This is (arguably) semi-supervised learning.


## Modeling scenarios



### Sequence modeling

* In NLP, language modeling is often used to model the probability of sequences of words (discrete variables).
* Useful in speech recognition and machine translation, with Bayes' rule, to score potential transcriptions and translations, respectively.
  * $p(English \mid French) \propto p(English) p(French \mid English)$
* Claude Shannon talked about this task [in the 40s](https://en.wikipedia.org/wiki/A_Mathematical_Theory_of_Communication).
* Applications for continuous data are also possible.
  * e.g., other types of time series (just think of a sentence as a time series with discrete events).



### Classification and regression

* Use RNN output and feed it into a classifier or regressor.
  * Take the output from last time step? From all steps (with attention or pooling to aggregate)?


### Transduction

* We might want to produce an output for each input.
  * e.g., POS Tagging ("The dog ran in the park" -> "article noun verb preposition article noun")




### Sequence to Sequence learning

* We may want to perform more complex transformations from one sequence to another.
  * e.g., machine translation, paraphrase, summarization
* [One possible method](https://arxiv.org/pdf/1409.3215v3.pdf) (a bit outdated):
  1. read inputs (e.g., English words) into one RNN.
  2. take the state and feed it into a second RNN.
  3. generate outputs (e.g., French words) from the second RNN until reaching a stop indicator.


### Reinforcement learning

* RNNs could be useful for RL when rewards are partially observable (e.g., [this paper](https://arxiv.org/pdf/1507.06527.pdf)).
 

## A few Ideas for Non Text Applications

* modeling how media consumption patterns (e.g., TV ratings) change over time
  * This is a bit like language modeling, but with ratings values instead of words.
* Sensor data (#IoT)
  * e.g., categorize signals
  * e.g., identify patterns within a longer signal
* Resource allocation
  * e.g., given jobs submitted to a work queue, we could identify when to allocate more workers/resources.
* App/website logs
  * e.g., cluster or categorize user behavior
* ...

## Some important deep learning concepts I haven't talked about

* [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) or regularization to avoid overfitting
* [initialization schemes](http://deepdish.io/2015/02/24/network-initialization/)
* activation functions (e.g., [ReLUs](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))
* [attention](https://arxiv.org/abs/1409.0473) for sequence-to-sequence problems (and classification, etc.)
* optimization with gradient descent (e.g., [Adam algorithm](https://arxiv.org/abs/1412.6980))


## Alternative methods for modeling sequences

* [Probabilistic graphical models](http://homes.cs.washington.edu/~taskar/pubs/gms-srl07.pdf) over sequences
  * undirected: [conditional random fields (CRFs)](https://en.wikipedia.org/wiki/Conditional_random_field)
  * directed: [hidden Markov models (HMMs)](http://mlg.eng.cam.ac.uk/zoubin/papers/ijprai.pdf)
* [discriminative Markov models](http://www.aclweb.org/anthology/W02-1001)
  * Basically, one uses a standard classifier to predict a label for each time step using the previous prediction along with the current input.
* The lines between these things and RNNs can be [kind of blurry](http://www.robots.ox.ac.uk/~szheng/papers/CRFasRNN.pdf).

## More Links

* [Yoav Goldberg's "A Primer on Neural Network Models for Natural Language Processing"](https://arxiv.org/abs/1510.00726)
* [Neural Network Zoo](http://www.asimovinstitute.org/neural-network-zoo/)
* [WildML's posts on RNNs](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
* [TensorFlow docs](https://www.tensorflow.org/versions/r0.11/tutorials/index.html)
* [paper on Gated RNNs](https://arxiv.org/abs/1412.3555)
* [Colah's "Understanding LSTM Networks"](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Thanks to various folks at Civis for discussion!